<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0101ENSkillsNetwork20718188-2022-01-01"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Regression Models with Keras</font></h1>


## Introduction


Despite the popularity of more powerful libraries such as PyToch and TensorFlow, they are not easy to use and have a steep learning curve. So, for people who are just starting to learn deep learning, there is no better library to use other than the Keras library.

Keras is a high-level API for building deep learning models. It has gained favor for its ease of use and syntactic simplicity facilitating fast development. As you will see in this lab and the other labs in this course, building a very complex deep learning network can be achieved with Keras with only few lines of code. You will appreciate Keras even more, once you learn how to build deep models using PyTorch and TensorFlow in the other courses.

<h2>Regression Models with Keras</h2>

<h3>Objective for this Notebook<h3>    
<h5> 1. How to use the Keras library to build a regression model.</h5>
<h5> 2. Download and Clean dataset </h5>
<h5> 3. Build a Neural Network </h5>
<h5> 4. Train and Test the Network. </h5>     


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="https://#item31">Download and Clean Dataset</a>
2.  <a href="https://#item32">Import Keras</a>
3.  <a href="https://#item33">Build a Neural Network</a>
4.  <a href="https://#item34">Train and Test the Network</a>

</font>
</div>


## Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [1]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented. 
# If you run this notebook on a different environment, e.g. your desktop, you may need to uncomment and install certain libraries.

#!pip install numpy==1.21.4
#!pip install pandas==1.3.4
#!pip install keras==2.1.6

In [2]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's download the data and read it into a <em>pandas</em> dataframe.


In [4]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa.


#### Let's check how many data points we have.


In [5]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [6]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [7]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [8]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

predictors_norm = (predictors - predictors.mean()) / predictors.std() #Normalizing the data
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [9]:
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [11]:
n_cols = predictors_norm.shape[1] # number of predictors
print("Number of columns:", n_cols)

Number of columns: 8


## Import Keras


In [12]:
import keras

As you can see, the TensorFlow backend was used to install the Keras library.


Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.


In [13]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split  # Splitting the data for train and test using Scikit-Learn
from sklearn.metrics import mean_squared_error        # Library to calculate the squared error between y_true and y_predict

<a id='item33'></a>


## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [14]:
# define regression model with 3 hidden layers of 10 neurons
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(n_cols, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has two hidden layers, each of 50 hidden units.


## Train and Test the Network


Let's call the function now to create our model.


In [15]:
# build the model
model = regression_model()
#Initialize the list to store 50 values of mean error squared 
mse = []

# Repeat 50 times
for i in range(0, 50):
 # Splitting data into 70% for training and 30% for testing
 X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.30)
 #Fitting the model with 50 epochs
 model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=50, verbose=2)
 #Predicting the value using test dataset to calculate the y_pred
 y_pred = model.predict(X_test)
 #Calculating mean squared error between y_true and y_pred
 error = mean_squared_error(y_test, y_pred)
 #Inserting the mean squared error for each time until 50 times
 mse.append(error)

Epoch 1/50
23/23 - 1s - loss: 1565.9039 - val_loss: 1548.7268 - 1s/epoch - 46ms/step
Epoch 2/50
23/23 - 0s - loss: 1544.1583 - val_loss: 1525.2794 - 84ms/epoch - 4ms/step
Epoch 3/50
23/23 - 0s - loss: 1515.8145 - val_loss: 1482.8140 - 75ms/epoch - 3ms/step
Epoch 4/50
23/23 - 0s - loss: 1457.1786 - val_loss: 1391.0721 - 83ms/epoch - 4ms/step
Epoch 5/50
23/23 - 0s - loss: 1332.1982 - val_loss: 1209.0428 - 84ms/epoch - 4ms/step
Epoch 6/50
23/23 - 0s - loss: 1107.6860 - val_loss: 907.4704 - 77ms/epoch - 3ms/step
Epoch 7/50
23/23 - 0s - loss: 774.8232 - val_loss: 543.3284 - 87ms/epoch - 4ms/step
Epoch 8/50
23/23 - 0s - loss: 435.2595 - val_loss: 306.7768 - 90ms/epoch - 4ms/step
Epoch 9/50
23/23 - 0s - loss: 272.5150 - val_loss: 285.3932 - 79ms/epoch - 3ms/step
Epoch 10/50
23/23 - 0s - loss: 240.0549 - val_loss: 264.7070 - 77ms/epoch - 3ms/step
Epoch 11/50
23/23 - 0s - loss: 222.4260 - val_loss: 247.5740 - 87ms/epoch - 4ms/step
Epoch 12/50
23/23 - 0s - loss: 209.5075 - val_loss: 234.1470 - 8

In [16]:
print(mse)

import statistics #Importing this library to calculate the Standard Deviation 

print("Mean of the error list:", np.mean(mse))

print("Standard Deviation of the error list:", statistics.stdev(mse))

[104.19824716048642, 71.51477568139396, 47.4050030933432, 48.00888914344371, 40.35374139141152, 36.6043373862079, 33.43961795756046, 35.17428361476626, 32.39084742426666, 30.170958965021917, 31.42223158386749, 26.79625652769815, 26.891421719790877, 27.69556380138313, 31.059716882381494, 30.326213599857205, 29.211712082439583, 26.87220145737171, 22.23436816700267, 28.002340447929473, 30.800806700626342, 25.7345118364234, 28.720400232473633, 24.035769322150507, 21.635957737103528, 24.30079153513345, 19.761157285293958, 25.74673954015739, 24.040880698337105, 24.475618961995796, 23.410117255050928, 20.196001249920577, 28.507820108121184, 24.238478711635064, 21.26908444645159, 23.799910106972362, 22.8759189890211, 20.1479509562711, 23.659516479992455, 18.003907218806138, 22.201067185961648, 17.265736942371447, 23.861067724372, 22.592786272334767, 21.083297749862947, 24.95354989908872, 25.32727318961423, 23.054445030016186, 22.110162503384924, 24.509405294274345]
Mean of the error list: 29.2

<hr>

Copyright © 2019 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_medium=dswb&utm_source=bducopyrightlink&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0101ENSkillsNetwork20718188-2022-01-01&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDL0101ENSkillsNetwork20718188-2022-01-01).
